In [1]:
!pip install gradio
!pip install langchain
!pip install transformers
!pip install faiss-cpu
!pip install openai
!pip install tiktoken
!pip install langchain-community langchain-core
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

In [2]:
!pip install  gradio


In [3]:

import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import torch
import gradio as gr


In [8]:
def load_and_process_pdf(pdf_path):
    try:

        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        docs = text_splitter.split_documents(documents[:55])
        return docs
    except Exception as e:
        return f"Error loading or processing PDF: {e}"


In [ ]:
# Loading the PDF
pdf_path = "/content/ConceptsofBiology-WEB.pdf"
docs = load_and_process_pdf(pdf_path)

# Training the Model
embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)
model_name = "google/flan-t5-base"
pipe = pipeline("text2text-generation", model=model_name, device=0 if torch.cuda.is_available() else -1)
llm = HuggingFacePipeline(pipeline=pipe)


# Q & A with the model
def create_qa_chain(docs, model_type):
    global vectorstore
    if model_type == "Hugging Face":

        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=vectorstore.as_retriever(),
            return_source_documents=True
        )


    else:
        return "Invalid model type. Please select 'TDIF' or 'Hugging Face'."
    return qa_chain

# Processing the Question
def answer_question( question, model_type):


    #print(docs[0])
    if isinstance(docs, str):
        return docs, ""

    qa = create_qa_chain(docs, model_type)
    if isinstance(qa, str):
        return qa, ""

    result = qa({"query": question.strip()})
    answer = result["result"]
    source_docs = "\n---\n".join([doc.page_content for doc in result["source_documents"]])
    return answer, source_docs

# User Interface with the Model

iface = gr.Interface(

    fn=answer_question,
    inputs=[

        gr.Textbox(label="Question"),
        gr.Radio(["Hugging Face"], label="Model Type")
    ],
    outputs=[gr.Textbox(label="Answer"), gr.Textbox(label="Source Documents")],
    title="RAG QA",
    description="Ask questions about your Concepts of Biology PDF."
)

iface.launch(debug=True)

<ipython-input-9-553366161eae>:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
Device set to use cpu


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f8313b29a59861db8d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Token indices sequence length is longer than the specified maximum sequence length for this model (822 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)
model_name = "google/flan-t5-base"
pipe = pipeline("text2text-generation", model=model_name, device=0 if torch.cuda.is_available() else -1)
llm = HuggingFacePipeline(pipeline=pipe)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True)

<ipython-input-10-1015760520d6>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (852 > 512). Running this sequence through the model will result in indexing errors
